---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [8]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [9]:
df=pd.read_fwf('university_towns.txt',header=None)
df.columns=['list']
state=df.loc[df['list'].str.split('\[').str[0].str.contains('\(')==False,'list'].str.split('\[').str[0]
region=df.loc[df['list'].str.split('\(').str[0].str.contains('\[')==False,'list'].str.split('\(').str[0]
check=pd.DataFrame(columns=['State','RegionName'])
i=0
for ind_value in state.index:
    #print(ind_value)
    if(state.index[-1]==ind_value):
        for j in range(ind_value+1,region.index[-1]+1):
            #print('{} is index for {}'.format(j,region[j]))
            tem=pd.Series({'State':state[ind_value],'RegionName':region[j]})
            check=check.append(tem,ignore_index=True)
    else:
        for j in range(ind_value+1,state.index[i+1]):
            #print('{} is index for {}'.format(j,region[j]))
            tem=pd.Series({'State':state[ind_value],'RegionName':region[j]})
            check=check.append(tem,ignore_index=True)
    i=i+1

check


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [10]:
data=pd.read_excel('gdplev.xls',skiprows=5)
data=data.iloc[214:,4:7]
data=data.iloc[:,[0,1]]
data.columns=['yearQ','GDP']
data=data.reset_index()
data=data.loc[:,['yearQ','GDP']]
data['diff']=data.loc[:,'GDP'].diff()
for i in range(len(data)-4):
    if(i==data.index[-3]):
        break
    if((data.loc[i,'GDP']>data.loc[i+1,'GDP'])&(data.loc[i+1,'GDP']>data.loc[i+2,'GDP'])&(data.loc[i+2,'GDP']<data.loc[i+3,'GDP'])&(data.loc[i+3,'GDP']<data.loc[i+4,'GDP'])):
        print(data.loc[i,'yearQ'])

2008q4


In [11]:
for i in range(len(data)-4):
    if(i==data.index[-3]):
        break
    if((data.loc[i,'GDP']>data.loc[i+1,'GDP'])&(data.loc[i+1,'GDP']>data.loc[i+2,'GDP'])&(data.loc[i+2,'GDP']<data.loc[i+3,'GDP'])&(data.loc[i+3,'GDP']<data.loc[i+4,'GDP'])):
        print(data.loc[i+4,'yearQ'])

2009q4


In [12]:
for i in range(len(data)-4):
    if(i==data.index[-3]):
        break
    if((data.loc[i,'GDP']>data.loc[i+1,'GDP'])&(data.loc[i+1,'GDP']>data.loc[i+2,'GDP'])&(data.loc[i+2,'GDP']<data.loc[i+3,'GDP'])&(data.loc[i+3,'GDP']<data.loc[i+4,'GDP'])):
        print(data.loc[i+2,'yearQ'])

2009q2


In [56]:
data=pd.read_csv('City_Zhvi_AllHomes.csv')
new_df=pd.DataFrame()
for i in range(2000,2017):
    new_df.loc[:,str(i)+'-q1']=(data.loc[:,str(i)+'-01']+data.loc[:,str(i)+'-02']+data.loc[:,str(i)+'-03'])/3
    new_df.loc[:,str(i)+'-q2']=(data.loc[:,str(i)+'-04']+data.loc[:,str(i)+'-05']+data.loc[:,str(i)+'-06'])/3
    new_df.loc[:,str(i)+'-q3']=(data.loc[:,str(i)+'-07']+data.loc[:,str(i)+'-08'])/3
    if (i==2016):
        break
    else:
        new_df.loc[:,str(i)+'-q4']=(data.loc[:,str(i)+'-10']+data.loc[:,str(i)+'-11']+data.loc[:,str(i)+'-12'])/3
new_df['RegionName']=data['RegionName']
abbr=pd.Series(states)
abbr=pd.DataFrame(abbr)
abbr=abbr.reset_index()
abbr.columns=['Abbr','State']
mer=pd.merge(data,abbr,how='inner',left_on='State',right_on='Abbr')
new_df['State']=mer['State_y']
new_df.set_index(['State','RegionName'],inplace=True)
new_df

2000-q1       2000-q2        2000-q3  \
State    RegionName                                                       
New York New York                      NaN           NaN            NaN   
         Los Angeles          2.070667e+05  2.144667e+05  146700.000000   
         Chicago              1.384000e+05  1.436333e+05   98200.000000   
         Philadelphia         5.300000e+04  5.363333e+04   35966.666667   
         Phoenix              1.118333e+05  1.143667e+05   77166.666667   
         Las Vegas            1.326000e+05  1.343667e+05   90066.666667   
         San Diego            2.229000e+05  2.343667e+05  162433.333333   
         Dallas               8.446667e+04  8.386667e+04   56466.666667   
         San Jose             3.742667e+05  4.065667e+05  285466.666667   
         Jacksonville         8.860000e+04  8.970000e+04   60966.666667   
         San Francisco        4.305000e+05  4.644667e+05  321233.333333   
         Austin               1.429667e+05  1.452667e+05   99233.333333   
         Detroit              6.616667e+04  6.830000e+04   44400.000000   
         Columbus             9.436667e+04  9.583333e+04   64633.333333   
         Memphis              7.250000e+04  7.320000e+04   49233.333333   
         Charlotte            1.269333e+05  1.283667e+05   86666.666667   
         El Paso              7.626667e+04  7.686667e+04   51100.000000   
         Boston               2.069333e+05  2.191667e+05  154066.666667   
         Seattle              2.486000e+05  2.556000e+05  174433.333333   
         Baltimore            5.966667e+04  5.950000e+04   39200.000000   
         Denver               1.622333e+05  1.678333e+05  115533.333333   
         Washington           1.377667e+05  1.442000e+05   99166.666667   
         Nashville            1.138333e+05  1.152667e+05   77100.000000   
         Milwaukee            7.803333e+04  7.906667e+04   53800.000000   
         Tucson               1.018333e+05  1.029667e+05   69466.666667   
         Portland             1.528000e+05  1.547667e+05  104333.333333   
         Oklahoma City        7.643333e+04  7.750000e+04   52300.000000   
         Omaha                1.128000e+05  1.141000e+05   77500.000000   
         Albuquerque          1.258667e+05  1.267000e+05   84300.000000   
         Fresno               9.410000e+04  9.526667e+04   64166.666667   
...                                    ...           ...            ...   
Vermont  Granite Shoals                NaN           NaN            NaN   
         Piney Point          1.556667e+05  1.551667e+05  105333.333333   
         Maribel                       NaN           NaN            NaN   
         Middleton            1.060667e+05  1.043333e+05   68133.333333   
         Bennett              1.329000e+05  1.358333e+05   92633.333333   
         East Hampstead       1.618333e+05  1.691000e+05  115600.000000   
         Garden City                   NaN           NaN            NaN   
         Mountainburg         5.716667e+04  6.433333e+04   45033.333333   
         Oostburg             1.072667e+05  1.081000e+05   74533.333333   
         Twin Peaks           9.736667e+04  1.001667e+05   67566.666667   
         Upper Brookville     1.230967e+06  1.230967e+06  823300.000000   
         Volcano              9.870000e+04  1.053667e+05   75300.000000   
         Wedgefield                    NaN           NaN            NaN   
         Williamston          1.591667e+05  1.613000e+05  109200.000000   
         Decatur              6.360000e+04  6.440000e+04   43533.333333   
         Briceville           4.000000e+04  4.173333e+04   29000.000000   
         Edgewood             9.170000e+04  9.186667e+04   61833.333333   
         Palmyra                       NaN           NaN            NaN   
         Saint Inigoes        1.480667e+05  1.476000e+05  103166.666667   
         Marysville                    NaN           NaN            NaN   
         Forest Falls         1.135333e+05  1.144000e+05   7